In [27]:
import pandas as pd
from pathlib import Path

In [28]:
stocks_df = pd.read_csv("../data/processed/cleaned_stock_data.csv")

In [ ]:
# Converting Date column to datetime format
stocks_df["Date"] = pd.to_datetime(stocks_df["Date"])

# Sorting data by Stock and Date to ensure proper time-series order
stocks_df = stocks_df.sort_values(["Stock", "Date"])

# Verifying sorting
stocks_df.head()


,Date,Open,High,Low,Close,Volume,Stock
2895,2000-01-03,150.924766,151.687436,148.058832,148.873622,2227434.0,AAPL
1147,2000-01-04,150.868034,151.828736,149.468681,150.037390,5362751.0,AAPL
717,2000-01-05,150.082163,150.729431,146.974248,148.832975,2253115.0,AAPL
2041,2000-01-11,150.062374,150.317808,147.630523,149.347065,5265559.0,AAPL
4149,2000-01-12,149.485782,149.779520,145.406565,146.694770,1119410.0,AAPL


In [30]:
# Checking missing values before cleaning
stocks_df.isnull().sum()

Date        0
Open      442
High      442
Low       442
Close     442
Volume    442
Stock       0
dtype: int64

In [31]:
# Dropping rows where all price-related columns are null
stocks_df = stocks_df.dropna(
    subset=["Open", "High", "Low", "Close"],
    how="all"
)

# Forward-filling missing price values within each stock
stocks_df[["Open", "High", "Low", "Close"]] = (
    stocks_df
    .groupby("Stock")[["Open", "High", "Low", "Close"]]
    .ffill()
)


In [32]:
# Filling missing Volume values using the median volume per stock
stocks_df["Volume"] = (
    stocks_df.groupby("Stock")["Volume"]
    .transform(lambda x: x.fillna(x.median()))
)

# Verifying missing values after cleaning
stocks_df.isnull().sum()


Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
Stock     0
dtype: int64

In [33]:
# Function to reindex each stock to continuous business days
def reindex_trading_days(df):
    # Create a complete range of business dates
    full_range = pd.date_range(
        start=df["Date"].min(),
        end=df["Date"].max(),
        freq="B"
    )
    
    # Reindex to include missing trading days
    return (
        df.set_index("Date")
          .reindex(full_range)
          .rename_axis("Date")
          .reset_index()
    )

# Apply reindexing for each stock
stocks_df = (
    stocks_df.groupby("Stock", group_keys=False)
    .apply(reindex_trading_days)
)

# Forward-fill values again after reindexing
stocks_df[["Open", "High", "Low", "Close", "Volume"]] = (
    stocks_df.groupby("Stock")[["Open", "High", "Low", "Close", "Volume"]]
    .ffill()
)


In [34]:
# Fixing Stock column after reindexing missing trading days
stocks_df["Stock"] = (
    stocks_df.groupby("Stock")["Stock"]
    .ffill()
)


In [35]:
# Calculating daily returns for each stock
stocks_df["Daily_Return"] = (
    stocks_df.groupby("Stock")["Close"].pct_change()
)


In [36]:
# Calculate Cumulative Return per stock
stocks_df["Cumulative_Return"] = (
    stocks_df.groupby("Stock")["Daily_Return"]
    .cumsum()
)


In [37]:
stocks_df[
    ["Date", "Stock", "Daily_Return", "Cumulative_Return"]
].head(15)


,Date,Stock,Daily_Return,Cumulative_Return
0,2000-01-03,AAPL,NaN,NaN
1,2000-01-04,AAPL,0.007817,0.007817
2,2000-01-05,AAPL,-0.008027,-0.000210
3,2000-01-06,NaN,NaN,NaN
4,2000-01-07,NaN,NaN,NaN
5,2000-01-10,NaN,NaN,NaN
6,2000-01-11,AAPL,0.003454,0.003244
7,2000-01-12,AAPL,-0.017759,-0.014515
8,2000-01-13,AAPL,0.002312,-0.012203
9,2000-01-14,NaN,NaN,NaN


In [38]:
# Calculate 20-day moving average
stocks_df["MA_20"] = (
    stocks_df.groupby("Stock")["Close"]
    .transform(lambda x: x.rolling(20).mean())
)

# Calculate 50-day moving average
stocks_df["MA_50"] = (
    stocks_df.groupby("Stock")["Close"]
    .transform(lambda x: x.rolling(50).mean())
)

# View final dataset
stocks_df.head()


,Date,Open,High,Low,Close,Volume,Stock,Daily_Return,Cumulative_Return,MA_20,MA_50
0,2000-01-03,150.924766,151.687436,148.058832,148.873622,2227434.0,AAPL,NaN,NaN,NaN,NaN
1,2000-01-04,150.868034,151.828736,149.468681,150.037390,5362751.0,AAPL,0.007817,0.007817,NaN,NaN
2,2000-01-05,150.082163,150.729431,146.974248,148.832975,2253115.0,AAPL,-0.008027,-0.000210,NaN,NaN
3,2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# Price Normalization
stocks_df["Normalized_Close"] = (
    stocks_df.groupby("Stock")["Close"]
    .transform(lambda x: (x - x.min()) / (x.max() - x.min()))
)


In [40]:
# Detect negative prices
invalid_prices = stocks_df[
    (stocks_df[["Open","High","Low","Close"]] < 0).any(axis=1)
]

if not invalid_prices.empty:
    print("⚠️ Warning: Invalid price records detected")
    print(invalid_prices[["Stock", "Date", "Open", "High", "Low", "Close"]].head())

    # Option 1: Drop invalid rows (recommended)
    stocks_df = stocks_df[
        (stocks_df[["Open","High","Low","Close"]] >= 0).all(axis=1)
    ]


⚠️ Warning: Invalid price records detected
     Stock       Date      Open      High       Low     Close
3809  MSFT 2014-08-08  1.632168  3.208556 -0.295409  1.661218
3811  MSFT 2014-08-12  0.612568  1.297806 -2.704839 -2.652708
3813  MSFT 2014-08-14 -2.576191 -1.986115 -6.893019 -6.032006
3814  MSFT 2014-08-15 -4.570030 -3.996917 -5.634444 -4.043480
3815  MSFT 2014-08-18 -3.921448  0.854705 -5.684744 -0.150174


In [41]:
# # File existence
# assert data.exists(), "Raw stock data folder missing"

# # Schema check
# required_cols = ["Date", "Open", "High", "Low", "Close", "Volume"]
# assert all(col in stocks_df.columns for col in required_cols), \
#     "Missing required stock columns"

# # Price sanity
# assert (stocks_df[["Open","High","Low","Close"]] >= 0).all().all(), \
#     "Negative price detected"

# # Volume sanity
# assert (stocks_df["Volume"] >= 0).all(), "Negative volume detected"



In [42]:
# Define output directory for processed data
output_path = Path("../data/processed")
output_path.mkdir(exist_ok=True)

# Save cleaned and enriched stock data
stocks_df.to_csv(
    output_path / "cleaned_stock_data.csv",
    index=False
)


In [43]:
transactions_df = pd.read_csv("../data/processed/cleaned_transactions.csv")

In [44]:
transactions_df.info()
transactions_df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Trade_Date  1000 non-null   object 
 1   Stock       1000 non-null   object 
 2   Quantity    857 non-null    float64
 3   Price       988 non-null    float64
 4   Trade_Type  695 non-null    object 
dtypes: float64(2), object(3)
memory usage: 39.2+ KB


Trade_Date      0
Stock           0
Quantity      143
Price          12
Trade_Type    305
dtype: int64

In [45]:
# Convert Trade_Date to datetime
transactions_df["Trade_Date"] = pd.to_datetime(
    transactions_df["Trade_Date"], errors="coerce"
)

# Keep only valid trade types
transactions_df = transactions_df[
    transactions_df["Trade_Type"].isin(["BUY", "SELL"])
]

# Remove rows with missing critical values
transactions_df = transactions_df.dropna(
    subset=["Trade_Date", "Stock", "Quantity", "Price"]
)

# Remove invalid quantities and prices
transactions_df = transactions_df[
    (transactions_df["Quantity"] > 0) &
    (transactions_df["Price"] > 0)
]

transactions_df.head()


,Trade_Date,Stock,Quantity,Price,Trade_Type
0,2005-06-07,GOOGL,25.0,296.133223,SELL
1,2008-03-10,GOOGL,50.0,1287.615079,SELL
4,2021-08-25,AAPL,10.0,1291.844776,BUY
6,2013-03-13,MSFT,10.0,847.886193,BUY
8,2008-11-07,AAPL,10.0,422.998729,BUY


In [46]:
# Save cleaned portfolio data
output_path = Path("../data/processed")
output_path.mkdir(exist_ok=True)

transactions_df.to_csv(
    output_path / "cleaned_transactions.csv",
    index=False
)
